In [1]:
import h5py
import numpy as np
from scipy.special import expit
import time

In [88]:
import h5py
with h5py.File('data-Mini Project 2.h5', 'r') as file:
    # List all the groups in the file
    trX = file['trX'][:]
    testdata = file['tstX'][:]
    trY = file['trY'][:]
    testlabel = file['tstY'][:]  


In [ ]:
class HAR:

    def initilaze(self,hidden_size,learning_rate):
        
        self.IH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, 4))
        self.HH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, hidden_size))
        self.OH_weight = np.random.uniform(-0.01, 0.01, size=(6, hidden_size+1))
        self.learning_rate = learning_rate
        self.hidden_size = hidden_size
            
        return
    
    def forward_propagation(self,datas,labels,batch_size):
        EPSILON = 1e-10
        grad_HO = 0
        cost = 0
        h_list = np.zeros((self.batch_size,1,self.hidden_size))
        input_list = np.zeros((self.batch_size,1,4))
        output_list = np.zeros((self.batch_size,1,6))  
        feedback=np.zeros((self.hidden_size,1))
        error=0.0
        correct_predictions=0
        for i in range(self.batch_size):
            data=datas[i].reshape(1,4)
            input_list[i]=data
            v_1= np.dot(self.IH_weight,data.T) + np.dot(self.HH_weight.T,feedback)
            h_i=self.Tanh_activation(v_1)
            feedback=h_i
            h_list[i]=feedback.T
            bias=-1 * np.ones((1, 1))
            biased_h_i=np.concatenate((h_i.T,bias),axis=1)
            y_input = biased_h_i
            v_2= np.dot(self.OH_weight,y_input.T)
            output=self.sigmoid_activation(v_2)
            labels=labels.reshape(6,1)
            correct_predictions += np.sum(np.argmax(output, axis=0) == np.argmax(labels, axis=0))
            grad_HO +=  np.dot((output-labels), biased_h_i)
            error += -labels * np.log10(np.clip(output, EPSILON, 1 - EPSILON)) -(1 - labels) * np.log10(np.clip(1 - output, EPSILON, 1 - EPSILON))

            output_list[i]=output.T

        return h_list,output_list,error,grad_HO,input_list,output,correct_predictions
        
        

    
    def gradient(self,y_n,d_n,h_list,x_list): #BPTT algorithm for updating WIH and WHH
        #h_list = [h0,h1,....,hn]
        h_n = h_list[-1]
    
        common = np.dot((y_n.T - d_n),self.OH_weight [:, :-1]) # 1x6 @ 6xN -> 1xN
    
        initial_term = common * (1-h_n**2)
    
        h_list = h_list[:-1] #Removing the h_n data from the list for convention
    
        index = 0
        grad_HH = np.zeros((self.hidden_size ,self.hidden_size))
        grad_IH = np.zeros((self.hidden_size,4))
        grad_bias1 = 0
    
        #Implementation of the derivative chain
        for i in reversed(range(1,len(h_list))): # i = n-1,n-2 .... , 0
            term = initial_term
            for j in range(len(h_list)-1,len(h_list)-index-1,-1): # j = n-1,....,n-1-index
    
                term = np.dot(term,self.HH_weight) * (1-h_list[j]**2) #term = (term @ WHH) * h[j]
    
            grad_HH += np.dot(term.T,h_list[i])
            #print("Delta Grad_HH:",np.dot((common * term).T,h_list[i]))
            grad_IH += np.dot( term.T,x_list[i])
            grad_bias1 += np.mean( term * -1)
            index += 1

        return grad_IH,grad_HH 
        
    
    def output_backward(self,dWoh):
        
        self.OH_weight= self.OH_weight + self.learning_rate*dWoh
    
    def hh_backward(self,gradient_list):
        self.HH_weight = self.HH_weight + self.learning_rate*gradient_list
        
    def IH_backward(self,gradient_list):
        self.IH_weight = self.IH_weight + self.learning_rate*gradient_list
        
    def Train(self,data,label,batch_size):
        startTime = time.time()
        correct_predictions2=0
        total=0
        correct_predictions3=0
        self.batch_size=batch_size
        for epoch in range(15):
            print("epoch: ", epoch + 1)
            cum_error=0.0
            
            correct_predictions2=0
            for idx in range(1500,2000):
                sample_data=data[idx,:,:]
                bias=-1 * np.ones((sample_data.shape[0], 1))
                biased_sample=np.hstack((sample_data,bias))
                self.batch_error=0.0
                correct_predictions1=0
                for time_idx in range(int(150/self.batch_size)):
                    total+=self.batch_size
                    datas=biased_sample[int(self.batch_size*time_idx):int(self.batch_size*(time_idx+1))]
                    labels=label[idx]
                    h_list,output_list,error,grad_HO,input_list,output,correct_predictions=self.forward_propagation(datas, labels,self.batch_size)
                    self.batch_error+=np.sum(error)
                    correct_predictions1+=correct_predictions
                    self.delta_weights=np.zeros((6, 50))                    
                    
                    grad_IH,grad_HH=self.gradient(output,labels,h_list,input_list)
                    max_gradient_norm = 0.5 
            
                    grad_HO_clipped = np.clip(grad_HO, -max_gradient_norm, max_gradient_norm)
                    grad_HH_clipped = np.clip(grad_HH, -max_gradient_norm, max_gradient_norm)
                    grad_IH_clipped = np.clip(grad_IH, -max_gradient_norm, max_gradient_norm)
                    self.OH_weight -= self.learning_rate * grad_HO/batch_size
                    self.HH_weight -= self.learning_rate * grad_HH_clipped/batch_size
                    self.IH_weight -= self.learning_rate * grad_IH_clipped/batch_size
                    
                #self.batch_error=(self.batch_error/batch_size)
                correct_predictions2+=correct_predictions1
                cum_error+=np.sum((self.batch_error/50))
                #print("Epoch error {i} {error}".format(i=epoch,error=batch_error))
            correct_predictions3+=correct_predictions2
            print("Error per epoch",cum_error)
        print("Train error %",100*correct_predictions3/total)

        
    def Tanh_activation(self,x):
        return np.tanh(x)
    
    def sigmoid_activation(self,x):
        y = expit(x)
        return y
    
    def Tanh_activation_derivative(self,x):
        return (1/2)*(1-x*x)
    
    def sigmoid_activation_derivative(self,x):
        return x-x*x


In [29]:
deneme1=HAR()
deneme1.initilaze(50,0.001)
deneme1.Train(trX,trY,10)

In [30]:
deneme1.Train(trX,trY,10)

epoch:  1
1950.5248699185024
epoch:  2
1610.6879933208184
epoch:  3
1564.5489289495458
epoch:  4
1544.0201927502824
epoch:  5
1518.5805863518503
epoch:  6
1475.980698401798
epoch:  7
1425.9856035754024
epoch:  8
1387.6667568529242
epoch:  9
1360.1865309432467
epoch:  10
1337.9654785427108
epoch:  11
1319.3357566487107
epoch:  12
1304.607414602222
epoch:  13
1293.8634890966014
epoch:  14
1286.4343596622869
epoch:  15
1281.7970699907896


In [90]:
def data_test(W_hh,W_oh,W_ih,hidden_size): 
    top1 = 0
    top2 = 0
    top3 = 0
    misclass = 0
    total = 0
    feedback = np.zeros((1,hidden_size))
    for test_idx in range(len(testdata)):
        test_data_idx = testdata[test_idx]
        for time_idx in range(150):
            test_reshaped = np.reshape(test_data_idx[time_idx], (1,3))
            biased_data = np.append(test_reshaped,-1)
            biased_data = np.reshape(biased_data, (1,4))#reshape biased data
            v_t = np.dot(biased_data, W_ih.T) + np.dot(feedback, W_hh.T)
            h_test = tanh(v_t)
            feedback = h_test
            biased_feedback = np.append(h_test,-1)
            v_ot = np.dot(biased_feedback, W_oh.T)
            output = sigmoid(v_ot)
            output = np.reshape(output, (1,6))
            desired = testlabel[test_idx]
            desired = np.reshape(desired, (1,6))
            output_list1 = output.copy()
            output_list1 = np.reshape(output_list1, np.shape(output))
            output_list1[0,np.argmax(output_list1)] = -9999999999999999
            output_list2 = output_list1.copy()
            output_list2 = np.reshape(output_list2, np.shape(output))
            output_list2[0,np.argmax(output_list2)] = -9999999999999999
            if np.argmax(output) == np.argmax(desired):
                total += 1
                top1 += 1
                top2 += 1
                top3 += 1
            elif np.argmax(output_list1) == np.argmax(desired):
                total += 1
                #print(oi_test)
                top2 += 1
                top3 += 1
            elif np.argmax(output_list2) == np.argmax(desired):
                total += 1
                top3 += 1
            else:
                total += 1
                misclass += 1 
    print("Top-1 Accuracy: %", 100*(top1/total))
    print("Top-2 Accuracy: %", 100*(top2/total))
    print("Top-3 Accuracy: %", 100*(top3/total))

In [31]:
W_hh=deneme1.HH_weight
W_oh =deneme1.OH_weight
W_ih =deneme1.IH_weight


# DATA SHUFFLE

In [172]:
permutation = np.random.permutation(trX.shape[0])
shuffled_train_data = trX
shuffled_labels = trY
for old, new in enumerate(permutation):
    shuffled_train_data[new,:,:] = trX[old,:,:]
    shuffled_labels[new,:] = trY[old,:]


In [13]:
def tanh(x):
    return np.tanh(x)

In [33]:
def sigmoid(x):
    return expit(x)

# CASE 1 N=50 BATCH 10 ,0.001

In [185]:
deneme1=HAR()
deneme1.initilaze(50,0.001)
deneme1.Train(trX,trY,10)

epoch:  1
Error per epoch 1940.3132243269904
epoch:  2
Error per epoch 1593.3346315128456
epoch:  3
Error per epoch 1547.2351086744854
epoch:  4
Error per epoch 1528.4298258524652
epoch:  5
Error per epoch 1507.7409714516004
epoch:  6
Error per epoch 1475.2579638034877
epoch:  7
Error per epoch 1437.0342434119375
epoch:  8
Error per epoch 1408.1719927551696
epoch:  9
Error per epoch 1388.9179088904132
epoch:  10
Error per epoch 1373.5041157467656
epoch:  11
Error per epoch 1359.602601467533
epoch:  12
Error per epoch 1347.2231923316424
epoch:  13
Error per epoch 1336.945229408092
epoch:  14
Error per epoch 1328.70823659169
epoch:  15
Error per epoch 1321.820344489645
Train error % 32.78305185185185


In [186]:
W_hh=deneme1.HH_weight
W_oh =deneme1.OH_weight
W_ih =deneme1.IH_weight

In [187]:
data_test(W_hh,W_oh,W_ih,50)

Top-1 Accuracy: % 36.08
Top-2 Accuracy: % 47.92111111111111
Top-3 Accuracy: % 61.577777777777776


# CASE 2 N=50 BATCH 10 ,0.005

In [176]:
deneme2=HAR()
deneme2.initilaze(50,0.005)
deneme2.Train(trX,trY,10)

epoch:  1
Error per epoch 1621.8849175879932
epoch:  2
Error per epoch 1418.057827026871
epoch:  3
Error per epoch 1328.9554127958272
epoch:  4
Error per epoch 1307.0557398230949
epoch:  5
Error per epoch 1348.657881609049
epoch:  6
Error per epoch 1280.3700787824341
epoch:  7
Error per epoch 1260.8777955878259
epoch:  8
Error per epoch 1215.828987453146
epoch:  9
Error per epoch 1192.5523268180889
epoch:  10
Error per epoch 1176.3022187086533
epoch:  11
Error per epoch 1154.112222812193
epoch:  12
Error per epoch 1165.7123897352474
epoch:  13
Error per epoch 1153.7867189039075
epoch:  14
Error per epoch 1141.0490663684927
epoch:  15
Error per epoch 1132.5750319011731
Train error % 37.92302222222222


In [177]:
data_test(deneme2.HH_weight,deneme2.OH_weight,deneme2.IH_weight,50)

Top-1 Accuracy: % 14.974444444444446
Top-2 Accuracy: % 33.452222222222225
Top-3 Accuracy: % 49.55777777777778


# CASE 3 N=50 BATCH 30 ,0.001

In [188]:
deneme3=HAR()
deneme3.initilaze(50,0.001)
deneme3.Train(trX,trY,30)

epoch:  1
Error per epoch 2278.8974677871724
epoch:  2
Error per epoch 1850.7387374591926
epoch:  3
Error per epoch 1694.583858089481
epoch:  4
Error per epoch 1625.4005656503614
epoch:  5
Error per epoch 1590.0484664975895
epoch:  6
Error per epoch 1569.9766139123933
epoch:  7
Error per epoch 1557.6102424464702
epoch:  8
Error per epoch 1549.463851821298
epoch:  9
Error per epoch 1543.774956172293
epoch:  10
Error per epoch 1539.580297816693
epoch:  11
Error per epoch 1536.316855359404
epoch:  12
Error per epoch 1533.6346812826223
epoch:  13
Error per epoch 1531.302787570074
epoch:  14
Error per epoch 1529.1589083333834
epoch:  15
Error per epoch 1527.0812670071848
Train error % 231.49653333333333


In [189]:
data_test(deneme3.HH_weight,deneme3.OH_weight,deneme3.IH_weight,50)

Top-1 Accuracy: % 16.666666666666664
Top-2 Accuracy: % 33.33333333333333
Top-3 Accuracy: % 50.0


# CASE 4 N=50 BATCH 30 ,0.005

In [180]:
deneme4=HAR()
deneme4.initilaze(50,0.005)
deneme4.Train(trX,trY,30)


epoch:  1
Error per epoch 1806.7996778131726
epoch:  2
Error per epoch 1551.6180916069538
epoch:  3
Error per epoch 1530.9721156725852
epoch:  4
Error per epoch 1518.5360379986118
epoch:  5
Error per epoch 1498.9588032939348
epoch:  6
Error per epoch 1467.8820669943323
epoch:  7
Error per epoch 1434.2708140661987
epoch:  8
Error per epoch 1409.5023843777483
epoch:  9
Error per epoch 1393.2019385031128
epoch:  10
Error per epoch 1380.8120085275923
epoch:  11
Error per epoch 1370.1913634241746
epoch:  12
Error per epoch 1361.143765672337
epoch:  13
Error per epoch 1354.0294315814097
epoch:  14
Error per epoch 1348.881296042328
epoch:  15
Error per epoch 1345.2262627075554
Train error % 295.088


In [181]:
data_test(deneme4.HH_weight,deneme4.OH_weight,deneme4.IH_weight,50)

Top-1 Accuracy: % 35.66777777777778
Top-2 Accuracy: % 47.43555555555555
Top-3 Accuracy: % 59.21666666666666


# CASE 5 N=100 BATCH 10 ,0.001

In [183]:
deneme5=HAR()
deneme5.initilaze(100,0.001)
deneme5.Train(trX,trY,10)


epoch:  1
Error per epoch 1940.4782160716336
epoch:  2
Error per epoch 1591.7860472366467
epoch:  3
Error per epoch 1544.0674167426935
epoch:  4
Error per epoch 1521.3313822051243
epoch:  5
Error per epoch 1493.1501908996602
epoch:  6
Error per epoch 1453.436715670124
epoch:  7
Error per epoch 1415.8412628519848
epoch:  8
Error per epoch 1389.9308848318187
epoch:  9
Error per epoch 1371.0342066120454
epoch:  10
Error per epoch 1356.2251941258278
epoch:  11
Error per epoch 1345.2782392147296
epoch:  12
Error per epoch 1337.598279630093
epoch:  13
Error per epoch 1331.808201057402
epoch:  14
Error per epoch 1326.8442281705163
epoch:  15
Error per epoch 1322.6330752592175
Train error % 33.477866666666664


In [184]:
data_test(deneme5.HH_weight,deneme5.OH_weight,deneme5.IH_weight,100)

Top-1 Accuracy: % 36.09222222222222
Top-2 Accuracy: % 48.19111111111111
Top-3 Accuracy: % 61.55666666666667


# CASE 6 N=100 BATCH 10 ,0.005

In [190]:
deneme6=HAR()
deneme6.initilaze(100,0.005)
deneme6.Train(trX,trY,10)


epoch:  1
Error per epoch 1613.5021356542456
epoch:  2
Error per epoch 1390.2855078976404
epoch:  3
Error per epoch 1327.2425567802304
epoch:  4
Error per epoch 1240.2092797308396
epoch:  5
Error per epoch 1107.9580006557865
epoch:  6
Error per epoch 1072.1294372650798
epoch:  7
Error per epoch 1066.0208207613493
epoch:  8
Error per epoch 1056.6769513352451
epoch:  9
Error per epoch 1046.0505763695983
epoch:  10
Error per epoch 1035.6190358225413
epoch:  11
Error per epoch 1025.9073216085535
epoch:  12
Error per epoch 1008.5360424772206
epoch:  13
Error per epoch 992.1891118108983
epoch:  14
Error per epoch 991.5417734640134
epoch:  15
Error per epoch 1005.3300613264378
Train error % 41.21078518518519


In [191]:
data_test(deneme6.HH_weight,deneme6.OH_weight,deneme6.IH_weight,100)

Top-1 Accuracy: % 16.634444444444444
Top-2 Accuracy: % 37.315555555555555
Top-3 Accuracy: % 50.0


# CASE 7 N=100 BATCH 30 ,0.001

In [192]:
deneme7=HAR()
deneme7.initilaze(100,0.001)
deneme7.Train(trX,trY,30)


epoch:  1
Error per epoch 2281.8821744287097
epoch:  2
Error per epoch 1850.8624513506875
epoch:  3
Error per epoch 1693.9284864065662
epoch:  4
Error per epoch 1624.4117884418824
epoch:  5
Error per epoch 1588.813328321387
epoch:  6
Error per epoch 1568.48680880142
epoch:  7
Error per epoch 1555.825500816089
epoch:  8
Error per epoch 1547.32842340314
epoch:  9
Error per epoch 1541.2212687666893
epoch:  10
Error per epoch 1536.5287396278052
epoch:  11
Error per epoch 1532.6743458513704
epoch:  12
Error per epoch 1529.2929429121496
epoch:  13
Error per epoch 1526.1367411762908
epoch:  14
Error per epoch 1523.0255278853626
epoch:  15
Error per epoch 1519.819292933373
Train error % 231.19946666666667


In [193]:
data_test(deneme7.HH_weight,deneme7.OH_weight,deneme7.IH_weight,100)

Top-1 Accuracy: % 16.666666666666664
Top-2 Accuracy: % 33.33333333333333
Top-3 Accuracy: % 50.0


# CASE 8 N=100 BATCH 30 ,0.005

In [194]:
deneme8=HAR()
deneme8.initilaze(100,0.005)
deneme8.Train(trX,trY,30)


epoch:  1
Error per epoch 1805.8289502502103
epoch:  2
Error per epoch 1550.2816194859645
epoch:  3
Error per epoch 1528.1988079122243
epoch:  4
Error per epoch 1512.7479159961695
epoch:  5
Error per epoch 1488.4468942663784
epoch:  6
Error per epoch 1453.6666340541199
epoch:  7
Error per epoch 1420.5713705328005
epoch:  8
Error per epoch 1397.6858941209548
epoch:  9
Error per epoch 1381.9218574657577
epoch:  10
Error per epoch 1370.0129577601256
epoch:  11
Error per epoch 1361.1439923494806
epoch:  12
Error per epoch 1354.8874844471134
epoch:  13
Error per epoch 1350.4820231674148
epoch:  14
Error per epoch 1347.1156328406191
epoch:  15
Error per epoch 1344.2476113159817
Train error % 300.96586666666667


In [195]:
data_test(deneme8.HH_weight,deneme8.OH_weight,deneme8.IH_weight,100)

Top-1 Accuracy: % 35.68333333333334
Top-2 Accuracy: % 47.06777777777778
Top-3 Accuracy: % 57.544444444444444


# FULL DATA BATCH 10 LR 0.001 HS=50

In [ ]:
class HAR2:

    def initilaze(self,hidden_size,learning_rate):
        
        self.IH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, 4))
        self.HH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, hidden_size))
        self.OH_weight = np.random.uniform(-0.01, 0.01, size=(6, hidden_size+1))
        self.learning_rate = learning_rate
        self.hidden_size = hidden_size
            
        return
    
    def forward_propagation(self,datas,labels,batch_size):
        EPSILON = 1e-10
        grad_HO = 0
        cost = 0
        h_list = np.zeros((self.batch_size,1,self.hidden_size))
        input_list = np.zeros((self.batch_size,1,4))
        output_list = np.zeros((self.batch_size,1,6))  
        feedback=np.zeros((self.hidden_size,1))
        error=0.0
        correct_predictions=0
        for i in range(self.batch_size):
            data=datas[i].reshape(1,4)
            input_list[i]=data
            v_1= np.dot(self.IH_weight,data.T) + np.dot(self.HH_weight.T,feedback)
            h_i=self.Tanh_activation(v_1)
            feedback=h_i
            h_list[i]=feedback.T
            bias=-1 * np.ones((1, 1))
            biased_h_i=np.concatenate((h_i.T,bias),axis=1)
            y_input = biased_h_i
            v_2= np.dot(self.OH_weight,y_input.T)
            output=self.sigmoid_activation(v_2)
            labels=labels.reshape(6,1)
            correct_predictions += np.sum(np.argmax(output, axis=0) == np.argmax(labels, axis=0))
            grad_HO +=  np.dot((output-labels), biased_h_i)
            error += -labels * np.log10(np.clip(output, EPSILON, 1 - EPSILON)) -(1 - labels) * np.log10(np.clip(1 - output, EPSILON, 1 - EPSILON))

            output_list[i]=output.T

        return h_list,output_list,error,grad_HO,input_list,output,correct_predictions
        
        

    
    def gradient(self,y_n,d_n,h_list,x_list): #BPTT algorithm for updating WIH and WHH
        #h_list = [h0,h1,....,hn]
        h_n = h_list[-1]
    
        common = np.dot((y_n.T - d_n),self.OH_weight [:, :-1]) # 1x6 @ 6xN -> 1xN
    
        initial_term = common * (1-h_n**2)
    
        h_list = h_list[:-1] #Removing the h_n data from the list for convention
    
        index = 0
        grad_HH = np.zeros((self.hidden_size ,self.hidden_size))
        grad_IH = np.zeros((self.hidden_size,4))
        grad_bias1 = 0
    
        #Implementation of the derivative chain
        for i in reversed(range(1,len(h_list))): # i = n-1,n-2 .... , 0
            term = initial_term
            for j in range(len(h_list)-1,len(h_list)-index-1,-1): # j = n-1,....,n-1-index
    
                term = np.dot(term,self.HH_weight) * (1-h_list[j]**2) #term = (term @ WHH) * h[j]
    
            grad_HH += np.dot(term.T,h_list[i])
            #print("Delta Grad_HH:",np.dot((common * term).T,h_list[i]))
            grad_IH += np.dot( term.T,x_list[i])
            grad_bias1 += np.mean( term * -1)
            index += 1

        return grad_IH,grad_HH 
        
    
    def output_backward(self,dWoh):
        
        self.OH_weight= self.OH_weight + self.learning_rate*dWoh
    
    def hh_backward(self,gradient_list):
        self.HH_weight = self.HH_weight + self.learning_rate*gradient_list
        
    def IH_backward(self,gradient_list):
        self.IH_weight = self.IH_weight + self.learning_rate*gradient_list
        
    def Train(self,data,label,batch_size):
        startTime = time.time()
        correct_predictions2=0
        total=0
        correct_predictions3=0
        self.batch_size=batch_size
        for epoch in range(15):
            print("epoch: ", epoch + 1)
            cum_error=0.0
            
            correct_predictions2=0
            for idx in range(3000):
                sample_data=data[idx,:,:]
                bias=-1 * np.ones((sample_data.shape[0], 1))
                biased_sample=np.hstack((sample_data,bias))
                self.batch_error=0.0
                correct_predictions1=0
                for time_idx in range(int(150/self.batch_size)):
                    total+=150/self.batch_size
                    datas=biased_sample[int(self.batch_size*time_idx):int(self.batch_size*(time_idx+1))]
                    labels=label[idx]
                    h_list,output_list,error,grad_HO,input_list,output,correct_predictions=self.forward_propagation(datas, labels,self.batch_size)
                    self.batch_error+=np.sum(error)
                    correct_predictions1+=correct_predictions
                    #print("Batch error {i} {error}".format(i=time_idx,error=error))
                    self.delta_weights=np.zeros((6, 50))                    
                    
                    grad_IH,grad_HH=self.gradient(output,labels,h_list,input_list)
                    #print(list_gradient[1].shape)
                    #grad_IH,grad_HH = self.calculate_grad(y_i,label,h_list,x_list)
                    max_gradient_norm = 0.5  # Adjust this threshold as needed
            
                    # Inside your weight update step
                    grad_HO_clipped = np.clip(grad_HO, -max_gradient_norm, max_gradient_norm)
                    grad_HH_clipped = np.clip(grad_HH, -max_gradient_norm, max_gradient_norm)
                    grad_IH_clipped = np.clip(grad_IH, -max_gradient_norm, max_gradient_norm)
            
                    #print("---------------- Gradients---------------")
                    #print("Sum of Unclipped Grad_HH:",np.sum(grad_HH),"Sum of Clipped Grad_HH:",np.sum(grad_HH_clipped))
                    #print("Sum of Unclipped Grad_IH:",np.sum(grad_IH),"Sum of Clipped Grad_IH:",np.sum(grad_IH_clipped))
            
                    #Updating the weights at the end of the batch
                    self.OH_weight -= self.learning_rate * grad_HO/batch_size
                    self.HH_weight -= self.learning_rate * grad_HH_clipped/batch_size
                    self.IH_weight -= self.learning_rate * grad_IH_clipped/batch_size
                    
                #self.batch_error=(self.batch_error/batch_size)
                correct_predictions2+=correct_predictions1
                cum_error+=np.sum((self.batch_error/50))
                #print("Epoch error {i} {error}".format(i=epoch,error=batch_error))
            correct_predictions3+=correct_predictions2
            print(cum_error)
        print("Train error",100*correct_predictions3/total)

        
    def Tanh_activation(self,x):
        return np.tanh(x)
    
    def sigmoid_activation(self,x):
        y = expit(x)
        return y
    
    def Tanh_activation_derivative(self,x):
        return (1/2)*(1-x*x)
    
    def sigmoid_activation_derivative(self,x):
        return x-x*x


In [196]:
deneme9=HAR2()
deneme9.initilaze(50,0.001)
deneme9.Train(trX,trY,10)


epoch:  1
9584.829579249212
epoch:  2
8177.657916647001
epoch:  3
7708.037334545171
epoch:  4
7548.950924847091
epoch:  5
7880.690275319436
epoch:  6


KeyboardInterrupt: 

In [ ]:
data_test(deneme9.HH_weight,deneme9.OH_weight,deneme9.IH_weight,50)

In [197]:
class HAR3:

    def initilaze(self,hidden_size,learning_rate):
        
        self.IH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, 4))
        self.HH_weight = np.random.uniform(-0.01, 0.01, size=(hidden_size, hidden_size))
        self.OH_weight = np.random.uniform(-0.01, 0.01, size=(6, hidden_size+1))
        self.learning_rate = learning_rate
        self.hidden_size = hidden_size
            
        return
    
    def forward_propagation(self,datas,labels,batch_size):
        EPSILON = 1e-10
        grad_HO = 0
        cost = 0
        h_list = np.zeros((self.batch_size,1,self.hidden_size))
        input_list = np.zeros((self.batch_size,1,4))
        output_list = np.zeros((self.batch_size,1,6))  
        feedback=np.zeros((self.hidden_size,1))
        error=0.0
        correct_predictions=0
        for i in range(self.batch_size):
            data=datas[i].reshape(1,4)
            input_list[i]=data
            v_1= np.dot(self.IH_weight,data.T) + np.dot(self.HH_weight.T,feedback)
            h_i=self.Tanh_activation(v_1)
            feedback=h_i
            h_list[i]=feedback.T
            bias=-1 * np.ones((1, 1))
            biased_h_i=np.concatenate((h_i.T,bias),axis=1)
            y_input = biased_h_i
            v_2= np.dot(self.OH_weight,y_input.T)
            output=self.sigmoid_activation(v_2)
            labels=labels.reshape(6,1)
            correct_predictions += np.sum(np.argmax(output, axis=0) == np.argmax(labels, axis=0))
            grad_HO +=  np.dot((output-labels), biased_h_i)
            error += -labels * np.log10(np.clip(output, EPSILON, 1 - EPSILON)) -(1 - labels) * np.log10(np.clip(1 - output, EPSILON, 1 - EPSILON))

            output_list[i]=output.T

        return h_list,output_list,error,grad_HO,input_list,output,correct_predictions
        
        

    
    def gradient(self,y_n,d_n,h_list,x_list): #BPTT algorithm for updating WIH and WHH
        #h_list = [h0,h1,....,hn]
        h_n = h_list[-1]
    
        common = np.dot((y_n.T - d_n),self.OH_weight [:, :-1]) # 1x6 @ 6xN -> 1xN
    
        initial_term = common * (1-h_n**2)
    
        h_list = h_list[:-1] #Removing the h_n data from the list for convention
    
        index = 0
        grad_HH = np.zeros((self.hidden_size ,self.hidden_size))
        grad_IH = np.zeros((self.hidden_size,4))
        grad_bias1 = 0
    
        #Implementation of the derivative chain
        for i in reversed(range(1,len(h_list))): # i = n-1,n-2 .... , 0
            term = initial_term
            for j in range(len(h_list)-1,len(h_list)-index-1,-1): # j = n-1,....,n-1-index
    
                term = np.dot(term,self.HH_weight) * (1-h_list[j]**2) #term = (term @ WHH) * h[j]
    
            grad_HH += np.dot(term.T,h_list[i])
            #print("Delta Grad_HH:",np.dot((common * term).T,h_list[i]))
            grad_IH += np.dot( term.T,x_list[i])
            grad_bias1 += np.mean( term * -1)
            index += 1

        return grad_IH,grad_HH 
        
    
    def output_backward(self,dWoh):
        
        self.OH_weight= self.OH_weight + self.learning_rate*dWoh
    
    def hh_backward(self,gradient_list):
        self.HH_weight = self.HH_weight + self.learning_rate*gradient_list
        
    def IH_backward(self,gradient_list):
        self.IH_weight = self.IH_weight + self.learning_rate*gradient_list
        
    def Train(self,data,label,batch_size):
        startTime = time.time()
        correct_predictions2=0
        total=0
        correct_predictions3=0
        self.batch_size=batch_size
        for epoch in range(15):
            print("epoch: ", epoch + 1)
            cum_error=0.0
            
            correct_predictions2=0
            for idx in range(3000):
                sample_data=data[idx,:,:]
                bias=-1 * np.ones((sample_data.shape[0], 1))
                biased_sample=np.hstack((sample_data,bias))
                self.batch_error=0.0
                correct_predictions1=0
                for time_idx in range(int(150/self.batch_size)):
                    total+=150/self.batch_size
                    datas=biased_sample[int(self.batch_size*time_idx):int(self.batch_size*(time_idx+1))]
                    labels=label[idx]
                    h_list,output_list,error,grad_HO,input_list,output,correct_predictions=self.forward_propagation(datas, labels,self.batch_size)
                    self.batch_error+=np.sum(error)
                    correct_predictions1+=correct_predictions
                    #print("Batch error {i} {error}".format(i=time_idx,error=error))
                    self.delta_weights=np.zeros((6, 50))                    
                    
                    grad_IH,grad_HH=self.gradient(output,labels,h_list,input_list)
                    #print(list_gradient[1].shape)
                    #grad_IH,grad_HH = self.calculate_grad(y_i,label,h_list,x_list)
                    max_gradient_norm = 0.5  # Adjust this threshold as needed
            
                    # Inside your weight update step
                    grad_HO_clipped = np.clip(grad_HO, -max_gradient_norm, max_gradient_norm)
                    grad_HH_clipped = np.clip(grad_HH, -max_gradient_norm, max_gradient_norm)
                    grad_IH_clipped = np.clip(grad_IH, -max_gradient_norm, max_gradient_norm)
            
                    #print("---------------- Gradients---------------")
                    #print("Sum of Unclipped Grad_HH:",np.sum(grad_HH),"Sum of Clipped Grad_HH:",np.sum(grad_HH_clipped))
                    #print("Sum of Unclipped Grad_IH:",np.sum(grad_IH),"Sum of Clipped Grad_IH:",np.sum(grad_IH_clipped))
            
                    #Updating the weights at the end of the batch
                    self.OH_weight -= self.learning_rate * grad_HO/batch_size
                    self.HH_weight -= self.learning_rate * grad_HH_clipped/batch_size
                    self.IH_weight -= self.learning_rate * grad_IH_clipped/batch_size
                    
                #self.batch_error=(self.batch_error/batch_size)
                correct_predictions2+=correct_predictions1
                cum_error+=np.sum((self.batch_error/50))
                #print("Epoch error {i} {error}".format(i=epoch,error=batch_error))
            correct_predictions3+=correct_predictions2
            print(cum_error)
        print("Train error",100*correct_predictions3/total)

        
    def Tanh_activation(self,x):
        return np.tanh(x)
    
    def sigmoid_activation(self,x):
        y = expit(x)
        return y
    
    def Tanh_activation_derivative(self,x):
        return (1/2)*(1-x*x)
    
    def sigmoid_activation_derivative(self,x):
        return x-x*x


In [163]:
deneme10=HAR3()
deneme10.initilaze(50,0.001)
deneme10.Train(trX,trY,10)


epoch:  1
1933.5060357555421
epoch:  2
1581.4346947841432
epoch:  3
1533.266350840202
epoch:  4
1514.3974871418202
epoch:  5
1495.8431825126845
epoch:  6
1467.2247605927225
epoch:  7
1431.202616376623
epoch:  8
1402.1424229263137
epoch:  9
1384.1146598197502
epoch:  10
1371.9087695859712
epoch:  11
1361.8922793871648
epoch:  12
1352.593348586671
epoch:  13
1343.684212733349
epoch:  14
1335.4319066996782
epoch:  15
1328.1647720618726
Train error 32.37837037037037


In [164]:
data_test(deneme10.HH_weight,deneme10.OH_weight,deneme10.IH_weight,50)

Top-1 Accuracy: % 35.05444444444444
Top-2 Accuracy: % 46.16888888888889
Top-3 Accuracy: % 65.17333333333333


# Overfitting find

In [198]:
deneme11=HAR3()
deneme11.initilaze(50,0.005)
deneme11.Train(trX,trY,10)


epoch:  1
8163.008148929671
epoch:  2
7208.699751090073
epoch:  3
7803.879650048873
epoch:  4
7950.256029598205
epoch:  5
7008.193046370538
epoch:  6
6823.833464930976
epoch:  7
6622.357222655244
epoch:  8
6524.917829049795
epoch:  9
6599.113355228023
epoch:  10
6563.621184366706
epoch:  11
6498.4357889587845
epoch:  12
6561.0906574474875
epoch:  13
6731.450833034972
epoch:  14
6756.77434603831
epoch:  15
6711.932075913851
Train error 40.22910617283951


In [199]:
data_test(deneme11.HH_weight,deneme11.OH_weight,deneme11.IH_weight,50)

Top-1 Accuracy: % 18.32777777777778
Top-2 Accuracy: % 51.14222222222222
Top-3 Accuracy: % 64.4211111111111


In [200]:
deneme12=HAR3()
deneme12.initilaze(50,0.001)
deneme12.Train(trX,trY,10)


epoch:  1
9571.126465790072
epoch:  2
8192.465041327856
epoch:  3
7764.4040843774965
epoch:  4
7574.158162506186
epoch:  5
8111.579809643991
epoch:  6
8355.290198863346
epoch:  7
8057.638361032556
epoch:  8
8234.725056320762
epoch:  9
8106.274454641604
epoch:  10
7969.076744208654
epoch:  11
7824.971804967103
epoch:  12
7714.185126898147
epoch:  13
7652.900359653235
epoch:  14
7638.61429791431
epoch:  15
7706.28827649167
Train error 35.54038518518519


In [201]:
data_test(deneme12.HH_weight,deneme12.OH_weight,deneme12.IH_weight,50)

Top-1 Accuracy: % 17.855555555555554
Top-2 Accuracy: % 32.92333333333333
Top-3 Accuracy: % 51.38333333333334


In [202]:
deneme13=HAR3()
deneme13.initilaze(50,0.001)
deneme13.Train(trX,trY,30)


epoch:  1
10614.895717846639
epoch:  2
9239.493269763365
epoch:  3
9118.419076988264
epoch:  4
8980.991832961985
epoch:  5
8732.315554388031
epoch:  6
8461.362991134773
epoch:  7
8281.597826461373
epoch:  8
8167.649888692043
epoch:  9
8079.6180648838545
epoch:  10
8002.03037248684
epoch:  11
7932.784255177374
epoch:  12
7874.534110138276
epoch:  13
7827.816941936407
epoch:  14
7789.504286651167
epoch:  15
7755.369006162909
Train error 302.6431111111111


In [203]:
data_test(deneme13.HH_weight,deneme13.OH_weight,deneme13.IH_weight,50)

Top-1 Accuracy: % 37.45777777777778
Top-2 Accuracy: % 47.69555555555556
Top-3 Accuracy: % 65.21111111111111


In [204]:
deneme14=HAR3()
deneme14.initilaze(50,0.005)
deneme14.Train(trX,trY,30)


epoch:  1
9356.055672929162
epoch:  2
8217.73491389517
epoch:  3
7829.118776064981
epoch:  4
7598.173791969922
epoch:  5
7378.552246630363
epoch:  6
7991.925600442427
epoch:  7
7512.698766233028
epoch:  8
7259.147429001301
epoch:  9
7146.927730427255
epoch:  10
7014.674958512691
epoch:  11
7041.126390609351
epoch:  12
7709.44593348968
epoch:  13
7311.65610449664
epoch:  14
7129.092605411075
epoch:  15
7061.724644916814
Train error 339.8813333333333


In [205]:
data_test(deneme14.HH_weight,deneme14.OH_weight,deneme14.IH_weight,50)

Top-1 Accuracy: % 16.924444444444443
Top-2 Accuracy: % 33.33333333333333
Top-3 Accuracy: % 50.0


In [206]:
deneme15=HAR3()
deneme15.initilaze(100,0.001)
deneme15.Train(trX,trY,10)


epoch:  1
9513.672442817233
epoch:  2
8078.514955509418
epoch:  3
7735.556254227175
epoch:  4
7848.689256585519
epoch:  5
7685.568495213353
epoch:  6
7530.616743321824
epoch:  7
7558.908799530256
epoch:  8
7431.650792192024
epoch:  9
7248.842426726276
epoch:  10
7115.502947330931
epoch:  11
7100.58043379533
epoch:  12
7037.946408711767
epoch:  13
6920.852722499561
epoch:  14
7022.0751423666115
epoch:  15
7252.870212584469
Train error 38.078064197530864


In [208]:
data_test(deneme15.HH_weight,deneme15.OH_weight,deneme15.IH_weight,100)

Top-1 Accuracy: % 16.668888888888887
Top-2 Accuracy: % 33.336666666666666
Top-3 Accuracy: % 50.00444444444444


In [212]:
deneme16=HAR3()
deneme16.initilaze(100,0.005)
deneme16.Train(trX,trY,30)


epoch:  1
9266.94479573958
epoch:  2
8118.0748028522885
epoch:  3
7805.884257173857
epoch:  4
7652.354318514225
epoch:  5
7721.027592016223
epoch:  6
7407.9613363947965
epoch:  7
7166.021125395094
epoch:  8
7138.981352705312
epoch:  9
7215.860816527946
epoch:  10
7153.815744056676
epoch:  11
7075.185635012642
epoch:  12
7034.465281910015
epoch:  13
6970.1620108192565
epoch:  14
6919.405813229523
epoch:  15
7472.702986348557
Train error 344.7080888888889


In [213]:
data_test(deneme16.HH_weight,deneme16.OH_weight,deneme16.IH_weight,100)

Top-1 Accuracy: % 16.66777777777778
Top-2 Accuracy: % 38.49333333333333
Top-3 Accuracy: % 50.18666666666667


In [214]:
deneme17=HAR3()
deneme17.initilaze(100,0.001)
deneme17.Train(trX,trY,30)


epoch:  1
10605.869189450776
epoch:  2
9219.224204794715
epoch:  3
9057.605826763864
epoch:  4
8836.517812428438
epoch:  5
8536.148806929554
epoch:  6
8316.843666504901
epoch:  7
8186.571345446454
epoch:  8
8088.986513804356
epoch:  9
8004.532028960151
epoch:  10
7932.019636859201
epoch:  11
7874.796335526782
epoch:  12
7832.44348313446
epoch:  13
7800.336503444667
epoch:  14
7773.143390058928
epoch:  15
7747.17517128542
Train error 309.09884444444447


In [215]:
data_test(deneme17.HH_weight,deneme17.OH_weight,deneme17.IH_weight,100)

Top-1 Accuracy: % 37.230000000000004
Top-2 Accuracy: % 48.11111111111111
Top-3 Accuracy: % 65.07111111111111


In [217]:
deneme18=HAR3()
deneme18.initilaze(100,0.005)
deneme18.Train(trX,trY,10)


epoch:  1
8031.779009488543
epoch:  2
7105.367153023587
epoch:  3
7376.171920781407
epoch:  4
6258.336893022159
epoch:  5
6993.540905516558
epoch:  6
7895.153560308152
epoch:  7
7858.917686190844
epoch:  8
7926.743114194737
epoch:  9
8014.8292217242915
epoch:  10
7920.525309559502
epoch:  11
7924.311366271285
epoch:  12
7784.043346183163
epoch:  13
8051.037077564447
epoch:  14
8342.595831944724
epoch:  15
8210.691130649644
Train error 36.385432098765435


In [218]:
data_test(deneme18.HH_weight,deneme18.OH_weight,deneme18.IH_weight,100)

Top-1 Accuracy: % 19.851111111111113
Top-2 Accuracy: % 34.67333333333333
Top-3 Accuracy: % 52.32555555555556
